In [31]:
import numpy as np
import pandas as pd
import matplotlib as mpl
import seaborn as sns
import matplotlib.pyplot as plt
import math
import plotly

In [32]:
import argparse

from joblib import load

from rdkit import Chem
from rdkit.Chem import AllChem
from rdkit.Chem import DataStructs

In [33]:
def ecfc_molstring(molecule, radius=3, size=8192):
    arr = np.zeros((1,), dtype=int)
    DataStructs.ConvertToNumpyArray(
        AllChem.GetHashedMorganFingerprint(molecule, radius, size, useFeatures=False),
        arr,
    )
    return arr


def smiles2mols(smiles):
    return [Chem.MolFromSmiles(smi) for smi in smiles]


def mols2features(mols):
    return np.array([ecfc_molstring(mol) for mol in mols])


def load_sklearn_model(model_path):
    return load(model_path)

In [34]:
df= pd.read_csv('train.csv', names=['smiles', 'logP'])
df

,smiles,logP
0,CCCC(=O)OCC(Cc1cncn1C)C(CC)C(=O)OCc1ccccc1,3.78
1,CCOc1ccccc1O,1.68
2,O=[N+]([O-])c1ccc(Oc2ccc(Cl)cc2Cl)cc1,4.64
3,Cc1cccc(C)n1,1.68
4,CC(=O)/C=C/C1C(C)=CCCC1(C)C,3.85
...,...,...
9995,CNC1CCc2c(OC)cccc2C1C,2.42
9996,Nc1ncc(Cc2cccc(Cl)c2Cl)c(N)n1,2.81
9997,c1ccc(N2CCCCC2)cc1,2.98
9998,CCCCCCN(SN(C)C(=O)O/N=C(\C)SC)C(=O)N(C)C,3.30


In [35]:
len(df)- len(df.drop_duplicates ())

0

In [36]:
df.describe()

,logP
count,10000.000000
mean,1.984328
std,1.799360
min,-11.960000
25%,0.810000
50%,1.940000
75%,3.100000
max,11.290000


In [37]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   smiles  10000 non-null  object 
 1   logP    10000 non-null  float64
dtypes: float64(1), object(1)
memory usage: 156.4+ KB


In [38]:
df['mol'] = df['smiles'].apply(lambda x: Chem.AddHs(Chem.MolFromSmiles(x)))
df['num_of_atoms'] = df['mol'].apply(lambda x: x.GetNumAtoms())
df['num_of_heavy_atoms'] = df['mol'].apply(lambda x: x.GetNumHeavyAtoms())

from rdkit.Chem import Descriptors
df['tpsa'] = df['mol'].apply(lambda x: Descriptors.TPSA(x))
df['mol_w'] = df['mol'].apply(lambda x: Descriptors.ExactMolWt(x))
df['num_valence_electrons'] = df['mol'].apply(lambda x: Descriptors.NumValenceElectrons(x))
df['num_heteroatoms'] = df['mol'].apply(lambda x: Descriptors.NumHeteroatoms(x))
df['num_rings'] = df['mol'].apply(lambda x: Descriptors.RingCount(x))
# df['Fp_Density_Morgan1'] = df['mol'].apply(lambda x: Descriptors.FpDensityMorgan1(x))
# df['Fp_Density_Morgan2'] = df['mol'].apply(lambda x: Descriptors.FpDensityMorgan2(x))
df['Fp_Density_Morgan3'] = df['mol'].apply(lambda x: Descriptors.FpDensityMorgan3(x))
df['logp_in'] = df['smiles'].apply(lambda x: Chem.rdMolDescriptors.CalcCrippenDescriptors(Chem.MolFromSmiles(x),includeHs=True)[0])

In [39]:
def number_of_atoms(atom_list, df):
    for i in atom_list:
        df['num_of_{}_atoms'.format(i)] = df['mol'].apply(lambda x: len(x.GetSubstructMatches(Chem.MolFromSmiles(i))))

In [40]:
symbols = ['C','N','O','F',
           'S','Cl', 'I']

In [41]:
def diff_atoms(df):
#     hetatms = set()
#     for i in range(len(df)):
#         mol = Chem.MolFromSmiles(df.iloc[0][0])
#         for atom in mol.GetAtoms():
#             hetatms.add(atom.GetSymbol())
    return symbols

In [42]:
number_of_atoms(diff_atoms(df), df)

In [43]:
features_test = pd.DataFrame(np.array([ecfc_molstring(m) for m in df['mol']]), index=None)

In [44]:
features_test.columns = features_test.columns.astype(str)

In [45]:
features_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Columns: 8192 entries, 0 to 8191
dtypes: int32(8192)
memory usage: 312.5 MB


In [46]:
df = pd.concat((df, features_test), axis=1)

In [47]:
df

,smiles,logP,mol,num_of_atoms,num_of_heavy_atoms,tpsa,mol_w,num_valence_electrons,num_heteroatoms,num_rings,...,8182,8183,8184,8185,8186,8187,8188,8189,8190,8191
0,CCCC(=O)OCC(Cc1cncn1C)C(CC)C(=O)OCc1ccccc1,3.78,<rdkit.Chem.rdchem.Mol object at 0x000002A8AE4...,58,28,70.42,386.220557,152,6,2,...,0,0,0,0,0,0,0,0,0,0
1,CCOc1ccccc1O,1.68,<rdkit.Chem.rdchem.Mol object at 0x000002A8AE4...,20,10,29.46,138.068080,54,2,1,...,0,0,0,0,0,0,0,0,0,0
2,O=[N+]([O-])c1ccc(Oc2ccc(Cl)cc2Cl)cc1,4.64,<rdkit.Chem.rdchem.Mol object at 0x000002A8AE4...,25,18,52.37,282.980298,92,6,2,...,0,0,0,0,0,0,0,0,0,0
3,Cc1cccc(C)n1,1.68,<rdkit.Chem.rdchem.Mol object at 0x000002A8AE4...,17,8,12.89,107.073499,42,1,1,...,0,0,0,0,0,0,0,0,0,0
4,CC(=O)/C=C/C1C(C)=CCCC1(C)C,3.85,<rdkit.Chem.rdchem.Mol object at 0x000002A8AE4...,34,14,17.07,192.151415,78,1,1,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,CNC1CCc2c(OC)cccc2C1C,2.42,<rdkit.Chem.rdchem.Mol object at 0x000002A8AE6...,34,15,21.26,205.146664,82,2,2,...,0,0,0,0,0,0,0,0,0,0
9996,Nc1ncc(Cc2cccc(Cl)c2Cl)c(N)n1,2.81,<rdkit.Chem.rdchem.Mol object at 0x000002A8AE6...,27,17,77.82,268.028252,88,6,2,...,0,0,0,0,0,0,0,1,0,0
9997,c1ccc(N2CCCCC2)cc1,2.98,<rdkit.Chem.rdchem.Mol object at 0x000002A8AE6...,27,12,3.24,161.120449,64,1,2,...,0,0,0,0,0,0,0,0,0,0
9998,CCCCCCN(SN(C)C(=O)O/N=C(\C)SC)C(=O)N(C)C,3.30,<rdkit.Chem.rdchem.Mol object at 0x000002A8AE6...,51,23,65.45,364.160283,134,9,0,...,0,0,0,0,0,0,0,0,0,0


In [48]:
df.describe()

,logP,num_of_atoms,num_of_heavy_atoms,tpsa,mol_w,num_valence_electrons,num_heteroatoms,num_rings,Fp_Density_Morgan3,logp_in,...,8182,8183,8184,8185,8186,8187,8188,8189,8190,8191
count,10000.000000,10000.000000,10000.00000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,...,10000.000000,10000.00000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000
mean,1.984328,32.456700,17.43830,61.135722,256.318411,93.678800,5.163700,1.769500,2.873610,2.037282,...,0.002800,0.00160,0.001900,0.006000,0.001400,0.001400,0.002800,0.010900,0.001500,0.002800
std,1.799360,15.835732,7.65653,38.561587,111.153605,41.644484,2.844101,1.185542,0.545302,1.577129,...,0.056502,0.03997,0.047923,0.100822,0.037392,0.042405,0.052844,0.108547,0.038703,0.063187
min,-11.960000,3.000000,1.00000,0.000000,16.031300,8.000000,0.000000,0.000000,0.500000,-5.395600,...,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.810000,22.000000,12.00000,34.140000,180.068748,66.000000,3.000000,1.000000,2.611111,1.006800,...,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,1.940000,29.000000,16.00000,55.760000,240.096837,88.000000,5.000000,2.000000,2.931034,1.962070,...,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,3.100000,39.000000,21.00000,82.280000,311.129207,112.000000,7.000000,2.000000,3.230769,2.984918,...,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,11.290000,193.000000,85.00000,288.280000,1176.784079,472.000000,22.000000,9.000000,4.600000,15.879200,...,2.000000,1.00000,2.000000,2.000000,1.000000,2.000000,1.000000,3.000000,1.000000,2.000000


In [49]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Columns: 8211 entries, smiles to 8191
dtypes: float64(5), int32(8192), int64(12), object(2)
memory usage: 313.9+ MB


In [50]:
# sns.heatmap(df.corr(numeric_only=True), annot=True, fmt=".1f")

In [51]:
from sklearn.metrics import mean_squared_error
def evaluation(model, X_test, y_test):
    prediction = model.predict(X_test)
    mse = math.sqrt(mean_squared_error(y_test, prediction))
    
    print('RMSE score:', round(mse,4))

## CatBoost

In [52]:
from catboost import CatBoostRegressor
from sklearn.model_selection import train_test_split

In [53]:
cb = CatBoostRegressor(depth=None, iterations=3900, learning_rate=0.1,
                        min_data_in_leaf=None, grow_policy='Lossguide', loss_function='RMSE', verbose=False)

In [54]:
X = df.drop(columns=['smiles', 'logP', 'mol'])
y = df['logP']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1)

In [55]:
cb.fit(X_train, y_train)

In [56]:
evaluation(cb, X_test, y_test)

RMSE score: 0.502


In [73]:
from joblib import dump, load
dump(cb, 'model.joblib') 

['model.joblib']

### xgboost 

In [57]:
from sklearn.preprocessing import LabelEncoder
from sklearn.impute import KNNImputer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import f1_score
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import cross_val_score

In [58]:
X = df.drop(columns=['smiles', 'logP', 'mol'])
y = df['logP']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1)

In [68]:
xgb_model = xgb.XGBRegressor()

In [69]:
xgb_model.fit(X_train, y_train)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=None, n_jobs=None,
             num_parallel_tree=None, random_state=None, ...)

In [71]:
evaluation(xgb_model, X_test, y_test)

RMSE score: 0.5461


In [72]:
from joblib import dump, load
dump(xgb_model, 'model.joblib') 

['model.joblib']